In [1]:
import pandas as pd
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image, ImageDraw
from sklearn.metrics import accuracy_score

class PoseDataset(Dataset):
    def __init__(self, data_folder, labels_file, transform=None):
        self.data_folder = data_folder
        self.labels_df = pd.read_csv(labels_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        file_id = self.labels_df.iloc[idx, 0]
        label = self.labels_df.iloc[idx, 1]
        data = np.load(os.path.join(self.data_folder, f'{file_id}.npy'))
        
        data = ((data - data.min()) * (255.0 / (data.max() - data.min()))).astype(np.uint8)
        
        image = Image.fromarray(data).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data_folder = 'C:/super ai/5domain/pose/train/train'
train_labels_file = 'C:/super ai/5domain/pose/train.csv'
train_dataset = PoseDataset(train_data_folder, train_labels_file, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model = models.resnet50(pretrained=True)
num_classes = 7  
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)

num_epochs = 20
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item()
    
    scheduler.step()
    
    current_lr = scheduler.get_last_lr()[0]
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, LR: {current_lr:.6f}')

c:\Users\_nut_natthawat_\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\_nut_natthawat_\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.data_files = [f for f in os.listdir(data_folder) if f.endswith('.npy')]
        self.transform = transform

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        file_id = self.data_files[idx].split('.')[0]
        data = np.load(os.path.join(self.data_folder, self.data_files[idx]))

        data = ((data - data.min()) * (255.0 / (data.max() - data.min()))).astype(np.uint8)

        image = Image.fromarray(data).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, file_id

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_data_folder = 'C:/super ai/5domain/pose/test/test'
test_dataset = TestDataset(test_data_folder, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()

results = []

with torch.no_grad():
    for images, file_ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        results.extend(zip(file_ids, predicted.cpu().numpy()))

submission_df = pd.DataFrame(results, columns=['id', 'class'])
submission_df.to_csv('submission_final_mai.csv', index=False)
print("Predictions saved to sample_submission.csv")

Predictions saved to sample_submission.csv
